# Imports

In [1]:
import sys
sys.path.append('../')
from mylibs import *

## Data
Load, split and prepare the data. Documentation of the functions can be found in data/data.py

In [2]:
from data.data import Data
handler = Data()

# load the data from pickle file one directory above the current directory
data = handler.load('../artifacts_5s_128hz.pkl')

# split dataset into train, val and test set
train_split, val_split, test_split = handler.split(data) 

# prepare balanced train dataset for convolutional lstm as tensorflow dataset
train = handler.prepare_data(train_split,
                             balance=True, 
                             dataset=True, 
                             lstm=True, 
                             ccn=True)


# prepare unbalanced validation dataset for convolutional lstm as tensorflow dataset
val = handler.prepare_data(val_split, 
                           dataset=True, 
                           lstm=True, 
                           ccn=True)

# prepare unbalanced test dataset for convolutional lstm as numpy arrays
test_features, test_labels = handler.prepare_data(test_split, 
                                                  lstm=True, 
                                                  ccn=True)

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



## Metrics

In [3]:
# all other metrics that we would like to track for each model architecture
metrics = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.CategoricalAccuracy(name='accuracy'),
      keras.metrics.CategoricalCrossentropy(name='loss'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

## Dynamic model building

In [4]:
from architectures.MultiLayerLSTM import MultiLayerLSTM
from architectures.BidirectionalLSTM import BidirectionalLSTM

def create_model(hparams):
    """
    returns a model based on the selected hyperparameters
    """
    models = {
        "MultiLayerLSTM": MultiLayerLSTM(num_hidden_units = hparams['HP_NUM_HIDDEN_UNITS'], 
                                        num_lstm_layers=hparams['HP_NUM_LSTM_LAYERS'],
                                        num_dense_units = hparams['HP_NUM_DENSE_LAYERS'],
                                        num_dense_layers = hparams['HP_NUM_DENSE_UNITS'],
                                        num_conv_layers=hparams['HP_NUM_CONV_LAYERS'],
                                        increase=hparams['HP_INCREASE_UNITS_PER_LSTM_LAYER']),
        
        "BidirectionalLSTM": BidirectionalLSTM(num_hidden_units=hparams['HP_NUM_HIDDEN_UNITS'],
                                             num_bidirectional_layers=hparams['HP_NUM_LSTM_LAYERS'],
                                             num_dense_units = hparams['HP_NUM_DENSE_LAYERS'],
                                             num_dense_layers = hparams['HP_NUM_DENSE_UNITS'],
                                             num_conv_layers=hparams['HP_NUM_CONV_LAYERS'],
                                             increase=hparams['HP_INCREASE_UNITS_PER_LSTM_LAYER'])
    }

    return models.get(hparams['HP_MODEL_ARCHITECTURE'])

## Dynamic Running

In [5]:
def run_experiment(hparams, logdir,savedir,checkpointdir,metrics):
    """
    builds, compiles, trains and evaluates a model with certain architectual hyperparameters

    Args:
        hparams: selected hyperparameters
        logdir: directory for logs
        savedir: directors for saving the model
        checkpointdir: direcotry for model checkpoints
        metrics: a list of metrics we want to track

    Returns:
        accuracy of trained model evaluated on test data set
    """
    model = create_model(hparams)
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=metrics)


    model.fit(train,
              batch_size=64,
              epochs=1,
              validation_data=val,
              verbose=0, # no output during training
              callbacks=[tf.keras.callbacks.TensorBoard(logdir),  # log metrics
                        hp.KerasCallback(logdir, hparams),  # log hparams
                        tf.keras.callbacks.ModelCheckpoint(filepath= os.path.join(checkpointdir, "ckpt_{epoch}") ,monitor='val_loss',save_weights_only=True), # save checkpoints when val loss goes down
                        keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)], # early stopping in the case that loss doesnt go down for 3 epochs
              ) 
    results = model.evaluate(test_features, test_labels)
    model.save(savedir)
    return results

## Dynamic Searching

In [6]:
def search(hparams_dict, logdir):
    """
    train models on the selected hyperparamers and log the metrics, checkpoints and trained model in logidr
    """
    session_num = 0
    for num_lstm_layers in hparams_dict['HP_NUM_LSTM_LAYERS'].domain.values:
        for num_hidden_units in hparams_dict['HP_NUM_HIDDEN_UNITS'].domain.values:
            for num_conv_layers in hparams_dict['HP_NUM_CONV_LAYERS'].domain.values:
                for num_dense_layers in hparams_dict['HP_NUM_DENSE_LAYERS'].domain.values:
                    for num_dense_units in hparams_dict['HP_NUM_DENSE_UNITS'].domain.values:
                        for increase in hparams_dict['HP_INCREASE_UNITS_PER_LSTM_LAYER'].domain.values:
                            for model_architecture in hparams_dict['HP_MODEL_ARCHITECTURE'].domain.values:
                                hparams = {
                                    'HP_MODEL_ARCHITECTURE': model_architecture,
                                    'HP_NUM_LSTM_LAYERS': num_lstm_layers,
                                    'HP_NUM_HIDDEN_UNITS': num_hidden_units,
                                    'HP_NUM_CONV_LAYERS': num_conv_layers,
                                    'HP_NUM_DENSE_LAYERS': num_dense_layers,
                                    'HP_NUM_DENSE_UNITS': num_dense_units,
                                    'HP_INCREASE_UNITS_PER_LSTM_LAYER': increase
                                }
                                run_name = "run-%d" % session_num
                                print('--- Starting trial: %s' % run_name)
                                print({h: hparams[h] for h in hparams})
                                results = run_experiment(logdir=logdir+'hparam_tuning/' + run_name, 
                                              hparams=hparams, 
                                              savedir=logdir+'models/'+run_name, 
                                              checkpointdir=logdir+'checkpoints'+run_name,
                                              metrics=metrics)
                                session_num += 1

## Search 1 
Plain MultiLayerLSTM and BidirectionalLSTM

In [25]:
HP_MODEL_ARCHITECTURE = hp.HParam('model_architecture', hp.Discrete(['MultiLayerLSTM', 'BidirectionalLSTM'])) # set model architecture as hyperparameter
HP_NUM_LSTM_LAYERS = hp.HParam('num_lstm_layers', hp.Discrete([1, 2, 4])) # try out 1, 2 or 4 layers for architecture
HP_NUM_HIDDEN_UNITS = hp.HParam('num_hidden_units', hp.Discrete([32, 64, 128, 256])) # try out different hidden_units
HP_NUM_DENSE_UNITS = hp.HParam('num_dense_units', hp.Discrete([0])) # try out different number of units for dense layer
HP_NUM_DENSE_LAYERS = hp.HParam('num_dense_layers', hp.Discrete([0])) # try out different number of dense layers
HP_INCREASE_UNITS_PER_LSTM_LAYER = hp.HParam('increase_units_per_lstm_layer', hp.Discrete([0])) # increase number of units per lstm layer

hparams_dict_1 = {
    'HP_MODEL_ARCHITECTURE': HP_MODEL_ARCHITECTURE,
    'HP_NUM_LSTM_LAYERS': HP_NUM_LSTM_LAYERS,
    'HP_NUM_HIDDEN_UNITS': HP_NUM_HIDDEN_UNITS,
    'HP_NUM_DENSE_UNITS': HP_NUM_DENSE_UNITS,
    'HP_NUM_DENSE_LAYERS': HP_NUM_DENSE_LAYERS,
    'HP_INCREASE_UNITS_PER_LSTM_LAYER': HP_INCREASE_UNITS_PER_LSTM_LAYER
}

search(logdir='logs1-1/', hparams_dict=hparams_dict_1)

HParam(name='model_architecture', domain=Discrete(['BidirectionalLSTM', 'MultiLayerLSTM']), display_name=None, description=None)

## Search 2
plain MultiLayerLSTM and BidirectionalLSTM but with increasing hidden units per lstm layer, every lstm layer get twice more hidden units than layer before

In [6]:
HP_MODEL_ARCHITECTURE = hp.HParam('model_architecture', hp.Discrete(['MultiLayerLSTM', 'BidirectionalLSTM'])) # set model architecture as hyperparameter
HP_NUM_LSTM_LAYERS = hp.HParam('num_lstm_layers', hp.Discrete([1, 2, 4])) # try out 1, 2 or 4 layers for architecture
HP_NUM_HIDDEN_UNITS = hp.HParam('num_hidden_units', hp.Discrete([32, 64, 128, 256])) # try out different hidden_units
HP_NUM_DENSE_UNITS = hp.HParam('num_dense_units', hp.Discrete([0])) # try out different number of units for dense layer
HP_NUM_DENSE_LAYERS = hp.HParam('num_dense_layers', hp.Discrete([0])) # try out different number of dense layers
HP_INCREASE_UNITS_PER_LSTM_LAYER = hp.HParam('increase_units_per_lstm_layer', hp.Discrete([1])) # increase number of units per lstm layer

hparams_dict_2 = {
    'HP_MODEL_ARCHITECTURE': HP_MODEL_ARCHITECTURE,
    'HP_NUM_LSTM_LAYERS': HP_NUM_LSTM_LAYERS,
    'HP_NUM_HIDDEN_UNITS': HP_NUM_HIDDEN_UNITS,
    'HP_NUM_DENSE_UNITS': HP_NUM_DENSE_UNITS,
    'HP_NUM_DENSE_LAYERS': HP_NUM_DENSE_LAYERS,
    'HP_INCREASE_UNITS_PER_LSTM_LAYER': HP_INCREASE_UNITS_PER_LSTM_LAYER
}

search(logdir='logs1-2/', hparams_dict=hparams_dict_2)

## Search 3

In [2]:
HP_MODEL_ARCHITECTURE = hp.HParam('model_architecture', hp.Discrete(['MultiLayerLSTM', 'BidirectionalLSTM'])) # set model architecture as hyperparameter
HP_NUM_LSTM_LAYERS = hp.HParam('num_lstm_layers', hp.Discrete([1, 2, 4])) # try out 1, 2 or 4 layers for architecture
HP_NUM_HIDDEN_UNITS = hp.HParam('num_hidden_units', hp.Discrete([32, 64, 128, 256])) # try out different hidden_units
HP_NUM_DENSE_UNITS = hp.HParam('num_dense_units', hp.Discrete([128, 256])) # try out different number of units for dense layer
HP_NUM_DENSE_LAYERS = hp.HParam('num_dense_layers', hp.Discrete([1,2,4])) # try out different number of dense layers
HP_INCREASE_UNITS_PER_LSTM_LAYER = hp.HParam('increase_units_per_lstm_layer', hp.Discrete([0])) # increase number of units per lstm layer

hparams_dict_3 = {
    'HP_MODEL_ARCHITECTURE': HP_MODEL_ARCHITECTURE,
    'HP_NUM_LSTM_LAYERS': HP_NUM_LSTM_LAYERS,
    'HP_NUM_HIDDEN_UNITS': HP_NUM_HIDDEN_UNITS,
    'HP_NUM_DENSE_UNITS': HP_NUM_DENSE_UNITS,
    'HP_NUM_DENSE_LAYERS': HP_NUM_DENSE_LAYERS,
    'HP_INCREASE_UNITS_PER_LSTM_LAYER': HP_INCREASE_UNITS_PER_LSTM_LAYER
}

search(logdir='logs1-2/', hparams_dict=hparams_dict_3)


## Search 4

In [ ]:
HP_MODEL_ARCHITECTURE = hp.HParam('model_architecture', hp.Discrete(['MultiLayerLSTM', 'BidirectionalLSTM'])) # set model architecture as hyperparameter
HP_NUM_LSTM_LAYERS = hp.HParam('num_lstm_layers', hp.Discrete([1, 2, 4])) # try out 1, 2 or 4 layers for architecture
HP_NUM_HIDDEN_UNITS = hp.HParam('num_hidden_units', hp.Discrete([32, 64, 128, 256])) # try out different hidden_units
HP_NUM_CONV_LAYERS = hp.HParam('num_conv_layers', hp.Discrete([1,2,3,4,5])) # try out different amounts of conv layers --> connected to filter size
HP_NUM_DENSE_UNITS = hp.HParam('num_dense_units', hp.Discrete([0])) # try out different number of units for dense layer
HP_NUM_DENSE_LAYERS = hp.HParam('num_dense_layers', hp.Discrete([0])) # try out different number of dense layers
HP_INCREASE_UNITS_PER_LSTM_LAYER = hp.HParam('increase_units_per_lstm_layer', hp.Discrete([0])) # increase number of units per lstm layer

hparams_dict_4 = {
    'HP_MODEL_ARCHITECTURE': HP_MODEL_ARCHITECTURE,
    'HP_NUM_LSTM_LAYERS': HP_NUM_LSTM_LAYERS,
    'HP_NUM_HIDDEN_UNITS': HP_NUM_HIDDEN_UNITS,
    'HP_NUM_CONV_LAYERS': HP_NUM_CONV_LAYERS,
    'HP_NUM_DENSE_UNITS': HP_NUM_DENSE_UNITS,
    'HP_NUM_DENSE_LAYERS': HP_NUM_DENSE_LAYERS,
    'HP_INCREASE_UNITS_PER_LSTM_LAYER': HP_INCREASE_UNITS_PER_LSTM_LAYER
}

search(logdir='logs1-4/', hparams_dict=hparams_dict_4)


In [7]:
HP_MODEL_ARCHITECTURE = hp.HParam('model_architecture', hp.Discrete(['BidirectionalLSTM'])) # set model architecture as hyperparameter
HP_NUM_LSTM_LAYERS = hp.HParam('num_lstm_layers', hp.Discrete([1, 2])) # try out 1, 2 or 4 layers for architecture
HP_NUM_HIDDEN_UNITS = hp.HParam('num_hidden_units', hp.Discrete([8,32])) # try out different hidden_units
HP_NUM_CONV_LAYERS = hp.HParam('num_conv_layers', hp.Discrete([1])) # try out different amounts of conv layers --> connected to filter size
HP_NUM_DENSE_UNITS = hp.HParam('num_dense_units', hp.Discrete([640])) # try out different number of units for dense layer
HP_NUM_DENSE_LAYERS = hp.HParam('num_dense_layers', hp.Discrete([0,1])) # try out different number of dense layers
HP_INCREASE_UNITS_PER_LSTM_LAYER = hp.HParam('increase_units_per_lstm_layer', hp.Discrete([0])) # increase number of units per lstm layer

hparams_dict_5 = {
    'HP_MODEL_ARCHITECTURE': HP_MODEL_ARCHITECTURE,
    'HP_NUM_LSTM_LAYERS': HP_NUM_LSTM_LAYERS,
    'HP_NUM_HIDDEN_UNITS': HP_NUM_HIDDEN_UNITS,
    'HP_NUM_CONV_LAYERS': HP_NUM_CONV_LAYERS,
    'HP_NUM_DENSE_UNITS': HP_NUM_DENSE_UNITS,
    'HP_NUM_DENSE_LAYERS': HP_NUM_DENSE_LAYERS,
    'HP_INCREASE_UNITS_PER_LSTM_LAYER': HP_INCREASE_UNITS_PER_LSTM_LAYER
}

search(logdir='logs1-5/', hparams_dict=hparams_dict_5)


--- Starting trial: run-0
{'HP_MODEL_ARCHITECTURE': 'BidirectionalLSTM', 'HP_NUM_LSTM_LAYERS': 1, 'HP_NUM_HIDDEN_UNITS': 8, 'HP_NUM_CONV_LAYERS': 1, 'HP_NUM_DENSE_LAYERS': 0, 'HP_NUM_DENSE_UNITS': 640, 'HP_INCREASE_UNITS_PER_LSTM_LAYER': 0}
Tensor("bidirectional_lstm/Cast:0", shape=(None, 640, 19, 1), dtype=float32)
Tensor("bidirectional_lstm/time_distributed_3/Reshape_1:0", shape=(None, 640, 17, 8), dtype=float32)
Tensor("bidirectional_lstm/time_distributed_4/Reshape_1:0", shape=(None, 640, 8, 8), dtype=float32)


KeyboardInterrupt: 